<a href="https://colab.research.google.com/github/Ricardo-Jaramillo/PySpark/blob/main/02_Missing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Missing Data

Let's check some of the most usefull methods to preprocess when missing data

In [1]:
# Install pyspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=5c8db6d35f22243da4a8565cf525311ccbb1b426766fb9866bb1ae63cc4b0b5a
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
# Import the Data
!wget https://raw.githubusercontent.com/Ricardo-Jaramillo/PySpark/main/datasets/ContainsNull.csv

--2023-10-03 00:24:49--  https://raw.githubusercontent.com/Ricardo-Jaramillo/PySpark/main/datasets/ContainsNull.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61 [text/plain]
Saving to: ‘ContainsNull.csv’

ContainsNull.csv    100%[===================>]      61  --.-KB/s    in 0s      

2023-10-03 00:24:49 (2.69 MB/s) - ‘ContainsNull.csv’ saved [61/61]



In [4]:
# Create a session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('miss').getOrCreate()

In [5]:
# Read in the data
df = spark.read.csv('ContainsNull.csv', header=True,
               inferSchema=True)

In [6]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp2| NULL| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



## Drop values

These are the attributes we can pass to `na.drop()` we'll use
* `thresh` int
* `how`
* `subset`


In [9]:
# Drop null valies
df.na.drop(thresh=2).show() # thresh drop any rows with at least a thresh number of non null values

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [11]:
# Drop ALL null valies
df.na.drop(how='any').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [12]:
# Drop rows that contains all null values
df.na.drop(how='any').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [13]:
# Drop rows that contain null values in the specified columns
df.na.drop(subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



## Now let's fill the missing data

In [14]:
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sales: double (nullable = true)



In [16]:
# Fill STRING columns with a STRING
df.na.fill('FILL VALUE').show()

+----+----------+-----+
|  Id|      Name|Sales|
+----+----------+-----+
|emp1|      John| NULL|
|emp2|FILL VALUE| NULL|
|emp3|FILL VALUE|345.0|
|emp4|     Cindy|456.0|
+----+----------+-----+



In [18]:
# Fill NUMERIC columns with a NUMERIC value
df.na.fill(0).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| NULL|  0.0|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [20]:
# Fill with a string specifying the column
df.na.fill('No Name', subset=['Name']).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| NULL|
|emp2|No Name| NULL|
|emp3|No Name|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+



## Fill with an aggregate value

In [28]:
# Import the mean function
from pyspark.sql.functions import mean

In [30]:
# Get the mean and stored it ina variable
mean_val = df.select(mean(df['Sales'])).collect()
mean_sales = mean_val[0][0]
mean_sales

400.5

In [31]:
# Fill it
df.na.fill(mean_sales, ['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| NULL|400.5|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [32]:
# Do all this in a line
df.na.fill(df.select(mean(df['Sales'])).collect()[0][0], ['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| NULL|400.5|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

